In [3]:
tree = etree.parse(dataset_directory + "/" + sample_input_files[0])
root = tree.getroot()

soup = BeautifulSoup(getxmlcontent(root))
text = soup.get_text()

doc = nlp(text)

sample = []
for ent in doc.ents:
    if ent.label_ == 'ORG':
        sample.append(ent.text)


NameError: name 'etree' is not defined

# Top 10 NER

This sample script will create two dataframes containing the top 10 entities as well as all entities that are found in a sample of your dataset.

In [1]:
# Importing all our required packages
import os
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
import random

# The spaCy library allows us to process and run operations on text
import spacy
from spacy.pipeline import EntityRecognizer

In [2]:
# Replace the path below with the dataset which you would like to use as input for the script
dataset_directory = '/home/ec2-user/SageMaker/Halfmann_Abortion'
input_files = os.listdir(dataset_directory)

In [3]:
sample_input_files = input_files

In [4]:
# Loading spacy's NLP model
nlp = spacy.load('en_core_web_sm')
# Setting max length to be larger than default so we can process longer articles
nlp.max_length = 15000000

In [5]:
# We define a function to get the text content that we need from the XML articles available in our dataset
def getxmlcontent(root):
    if root.find('.//HiddenText') is not None:
        return(root.find('.//HiddenText').text)
    elif root.find('.//Text') is not None:
        return(root.find('.//Text').text)
    else:
        print('Could not find \"HiddenText\" or \"Text\" tags')
        return None

In [7]:
# Creating empty dictionary in which to put our entities and counts
date = []
article_text = {}

for article in sample_input_files:
    try:
        # Parse data and get root tags
        tree = etree.parse(dataset_directory + "/" + article)
        root = tree.getroot()
        
        article_id = root.find("GOID").text
        article_text[article_id] = {}
        text = BeautifulSoup(getxmlcontent(root)).get_text()
        doc = nlp(text)
        sample = []
        for ent in doc.ents:
            if ent.label_ == "ORG":
                sample.append(ent.text)
        article_text[article_id]["text"] = sample
        for elem in root.iter("NumericDate"):
            article_text[article_id]["DATE"] = elem.text
                
    except TypeError:
        print('Type Error' + article)
        continue
    except AttributeError:
        print("Attriubte Error" + article)
        continue
        

SyntaxError: invalid or missing encoding declaration for '/home/ec2-user/SageMaker/Halfmann_Abortion/data15.csv' (<string>)

In [16]:
df_text = pd.DataFrame.from_dict(article_text, orient = "index")

In [21]:
df_text.to_csv("../document_ner.csv")

In [22]:
df_text.to_csv("../document_ner_compressed.csv", compression = "zip")

In [27]:
df_text

,text,DATE
118200528,"[More Liberal Society, Afraid of Virginia, the...",1968-04-29
109085647,"[House, Senate, ADAM CLYMER, The New York Time...",1993-03-12
2463412533,"[Interior, Orofino Idaho, Senate, Cessna, Cong...",2017-08-26
110844154,"[Cardinal, Roman Catholic Archdiocese, Cardina...",1987-01-24
108962491,"[The New York Times, Parliament, Parliament, W...",1992-11-05
...,...,...
118177575,"[irs, Odie, Courageous Cat, Islamic Center of ...",1967-02-26
108510026,"[PUBLIC & PRIVATE, The New York Times, the Apo...",1990-09-16
118956856,"[The New York Times, the American Medical Asso...",1970-06-23
1621335386,"[Health, icine, Iter, Iter, Mount Sinai Medica...",2011-08-14


In [28]:
df_text_copy = df_text.copy()

In [30]:
year = []
for i in range(1960,2018):
    year.append(str(i))

In [31]:
dataframe_year = []

for i in range(len(df_text_copy["DATE"])):
    for j in range(len(year)):
        if df_text_copy["DATE"].values[i][0:4] == year[j]:
            dataframe_year.append(year[j])

In [32]:
df_text_copy["YEAR"] = dataframe_year

In [33]:
df_text_copy1 = df_text_copy.copy()

In [44]:
#df2 = pd.DataFrame([x for x in df_text_copy1["text"].apply(lambda item: dict(map(lambda x: (x,1),item))).values]).fillna(0)
final = []
for i in range(len(df_text_copy1["text"].values)):
    final.append(",".join(df_text_copy1["text"].values[i]))

In [47]:
df_text_copy1["TEXT"] = final

In [55]:
df_text_copy2 = df_text_copy1[["YEAR","TEXT"]]

In [57]:
df_text_copy2.to_csv("ner_article.csv",compression="zip")

In [58]:
df_text_copy2.to_csv("ner_article_noncompressed_csv")

In [11]:
import csv

with open("out.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(entitydict)

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(entitydict), columns=mlb.classes_)

MemoryError: Unable to allocate 29.8 GiB for an array with shape (28247, 141460) and data type int64

In [8]:
# Creating the dataframes containing all entities as well as top 10 entities
entity_df = pd.DataFrame.from_dict(entitydict, orient='index')
top10_df = entity_df.sort_values(by=[0],ascending=False).head(10)

In [9]:
# View top 10 entities dataframe
top10_df

,0,1
Republican,36523,NORP
one,34354,CARDINAL
first,26549,ORDINAL
two,25620,CARDINAL
Senate,20605,ORG
Bush,20048,PERSON
Democrats,19645,NORP
Republicans,19481,NORP
Democratic,17715,NORP
today,16044,DATE


In [10]:
# View dataframe of all entities (will be truncated)
entity_df

,0,1
Today,2450,DATE
More Liberal Society,1,ORG
ROBERT E. DALLOS,1,PERSON
Jack,328,PERSON
Jill,26,PERSON
...,...,...
32.7-year-old,1,DATE
Jack* Kemp,1,PERSON
Social' Security,1,ORG
aDemocratic,1,GPE


In [19]:
entities = entity_df.sort_values(by=[0],ascending=False)
org = entities[entities[1] == "ORG"]
org = org[org[0] > 10]

In [23]:
org.to_csv("../organizations.csv")